### (같은 시간에 방송된 exp_min, total, sales를 합쳐서 가져오기)
* 같은 시간에 판매하며, 옵션이 다른 (예를 들면 여성용/남성용, 무이자/일시불, 10인용/6인용)도 제거하지 않고 각 각 노출시간 나오게 하기
* 방법 3을 선택해서 분석하였음

In [2]:
import pandas as pd

In [3]:
if 'google.colab' in str(get_ipython()):
    print('Running on CoLab')
    from google.colab import drive
    drive.mount('/content/drive')
    df = pd.read_excel('/content/drive/My Drive/빅데이터_콘테스트/base_line.xlsx', header=0)
else:
    print('Not running on CoLab')
    df = pd.read_excel('c:/bc/base_line.xlsx', header=0)

Not running on CoLab


In [4]:
# 방법1 => size로 해버리면 중복제거도 되고, exp_min도 나온 갯수 곱해서 최총 exp_min이 나올 수 있었지만, size()뒤에 다른것을 입력x

df['exp_min'] = df['exp_min'].fillna(method='ffill')# 결측값은 앞의 값으로 채우기
ded_df = df.groupby(['YEAR_DAY','p_group','m_code','p_name','exp_min']).size() # 중복은 제거한 목록 보이기 위해 

In [5]:
# 방법2 => agg함수를 찾았지만, index가 문제인거 같아 뒤에 reset_index()를 더했다.
#          숫자로 index가 매겨지며, 
#          multi_index가 되어 통계, plot 혹은 다른 함수들 사용할 때 번거로울거 같아 없애는 방법을 찾다가 도저히 모르겠다.
#          droplevel 후 , merge도 잘 안됐고, index를 바꾸려 해도 배열이 안맞다고 안됐다.

df['exp_min'] = df['exp_min'].fillna(method='ffill')# 결측값은 앞의 값으로 채우기
ded_df = df.groupby(['YEAR_DAY','p_group','m_code','p_name']).agg({'exp_min':['sum'],'unit_price':['min'], 'total_price':['sum'],'sales':['sum'],'no':['min']}).reset_index()

In [6]:
# 방법3 => reset_index를 하지않고, 생각했다. -> 최종선택
#          1. column 명을 1줄로 바꿨다. no를 가져온 이유는 나중에 merge할 때 쓰기 위해
#          2. 1개 index(YEAR_DAY)만 냅두고 제거할 변수(droplevel) 선택/ 왜냐면 우리에겐 no가 있으니깐 그걸로 merge하면 된다는 idea! 
#          3. merge로 완성하기

df['exp_min'] = df['exp_min'].fillna(method='ffill')# 결측값은 앞의 값으로 채우기
ded_df = df.groupby(['YEAR_DAY','p_group','m_code','p_name']).agg({'exp_min':['sum'],'unit_price':['min'], 'total_price':['sum'],'sales':['sum'],'no':['min']})

In [7]:
ded_df #multi_index로 만들었다. 아직은 분석하기도 어렵고, 함수 골라서 쓰기가 번거롭다. => 바꾸자

exp_min unit_price  \
                                                              sum        min   
YEAR_DAY   p_group m_code p_name                                               
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                     60.0     489000   
                          일시불 LG 통돌이 세탁기                     60.0     439000   
           농수축     100322 국내산 손질갑오징어 8팩                      40.0      49900   
                   100435 우리바다 손질왕꼬막 24팩                     60.0      39900   
                   100496 뽕셰프 이봉원 특갈비탕 12팩                   60.0      60900   
...                                                           ...        ...   
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                      60.0     148000   
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)    60.0     218000   
                          무이자 쿠쿠전기밥솥 6인용(QS)                 60.0     208000   
                          일시불  쿠쿠전기밥솥 6인용(QS)                60.0     198000   
                          일시불 쿠쿠전기밥솥 10인용 (QS)               60.0     208000   

                                                          total_price sales  \
                                                                  sum   sum   
YEAR_DAY   p_group m_code p_name                                              
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                    123876000   253   
                          일시불 LG 통돌이 세탁기                    160698000   367   
           농수축     100322 국내산 손질갑오징어 8팩                     176244000  3532   
                   100435 우리바다 손질왕꼬막 24팩                    246455000  6177   
                   100496 뽕셰프 이봉원 특갈비탕 12팩                   86177000  1415   
...                                                               ...   ...   
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                      58429000   395   
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)    73048000   335   
                          무이자 쿠쿠전기밥솥 6인용(QS)                 11882000    58   
                          일시불  쿠쿠전기밥솥 6인용(QS)                21743000   110   
                          일시불 쿠쿠전기밥솥 10인용 (QS)               49330000   238   

                                                              no  
                                                             min  
YEAR_DAY   p_group m_code p_name                                  
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                      45  
                          일시불 LG 통돌이 세탁기                      44  
           농수축     100322 국내산 손질갑오징어 8팩                       32  
                   100435 우리바다 손질왕꼬막 24팩                      38  
                   100496 뽕셰프 이봉원 특갈비탕 12팩                    29  
...                                                          ...  
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                    35370  
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)  35310  
                          무이자 쿠쿠전기밥솥 6인용(QS)               35312  
                          일시불  쿠쿠전기밥솥 6인용(QS)              35313  
                          일시불 쿠쿠전기밥솥 10인용 (QS)             35311  

[12510 rows x 5 columns]

In [8]:
#merge하거나 index를 다시 부여할 때 배열이 안맞다는 에러가 났다.-> 이름바꾸기
ded_df.columns = ['exp_min', 'unit_price', 'total_price','sales','no'] 
ded_df

exp_min  \
YEAR_DAY   p_group m_code p_name                                     
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                      60.0   
                          일시불 LG 통돌이 세탁기                      60.0   
           농수축     100322 국내산 손질갑오징어 8팩                       40.0   
                   100435 우리바다 손질왕꼬막 24팩                      60.0   
                   100496 뽕셰프 이봉원 특갈비탕 12팩                    60.0   
...                                                            ...   
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                       60.0   
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)     60.0   
                          무이자 쿠쿠전기밥솥 6인용(QS)                  60.0   
                          일시불  쿠쿠전기밥솥 6인용(QS)                 60.0   
                          일시불 쿠쿠전기밥솥 10인용 (QS)                60.0   

                                                           unit_price  \
YEAR_DAY   p_group m_code p_name                                        
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                       489000   
                          일시불 LG 통돌이 세탁기                       439000   
           농수축     100322 국내산 손질갑오징어 8팩                         49900   
                   100435 우리바다 손질왕꼬막 24팩                        39900   
                   100496 뽕셰프 이봉원 특갈비탕 12팩                      60900   
...                                                               ...   
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                        148000   
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)      218000   
                          무이자 쿠쿠전기밥솥 6인용(QS)                   208000   
                          일시불  쿠쿠전기밥솥 6인용(QS)                  198000   
                          일시불 쿠쿠전기밥솥 10인용 (QS)                 208000   

                                                           total_price  sales  \
YEAR_DAY   p_group m_code p_name                                                
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                     123876000    253   
                          일시불 LG 통돌이 세탁기                     160698000    367   
           농수축     100322 국내산 손질갑오징어 8팩                      176244000   3532   
                   100435 우리바다 손질왕꼬막 24팩                     246455000   6177   
                   100496 뽕셰프 이봉원 특갈비탕 12팩                    86177000   1415   
...                                                                ...    ...   
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                       58429000    395   
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)     73048000    335   
                          무이자 쿠쿠전기밥솥 6인용(QS)                  11882000     58   
                          일시불  쿠쿠전기밥솥 6인용(QS)                 21743000    110   
                          일시불 쿠쿠전기밥솥 10인용 (QS)                49330000    238   

                                                              no  
YEAR_DAY   p_group m_code p_name                                  
2019-01-01 가전      100150 무이자 LG 통돌이 세탁기                      45  
                          일시불 LG 통돌이 세탁기                      44  
           농수축     100322 국내산 손질갑오징어 8팩                       32  
                   100435 우리바다 손질왕꼬막 24팩                      38  
                   100496 뽕셰프 이봉원 특갈비탕 12팩                    29  
...                                                          ...  
2019-12-31 주방      100448 일시불쿠첸압력밥솥 6인용                    35370  
                   100837 무이자 쿠쿠전기밥솥 10인용(CRP-QS107FG/FS)  35310  
                          무이자 쿠쿠전기밥솥 6인용(QS)               35312  
                          일시불  쿠쿠전기밥솥 6인용(QS)              35313  
                          일시불 쿠쿠전기밥솥 10인용 (QS)             35311  

[12510 rows x 5 columns]

In [9]:
ded_df= ded_df.droplevel([1,2,3,]) #multi_index라서 삭제해줘야지 plot 그리거나 데이터 분석할 때 편하다고 생각했다. no가 있으니깐 ok

In [10]:
ded_df

,exp_min,unit_price,total_price,sales,no
YEAR_DAY,,,,,
2019-01-01,60.0,489000,123876000,253,45
2019-01-01,60.0,439000,160698000,367,44
2019-01-01,40.0,49900,176244000,3532,32
2019-01-01,60.0,39900,246455000,6177,38
2019-01-01,60.0,60900,86177000,1415,29
...,...,...,...,...,...
2019-12-31,60.0,148000,58429000,395,35370
2019-12-31,60.0,218000,73048000,335,35310
2019-12-31,60.0,208000,11882000,58,35312


In [13]:
ded_df= pd.merge(df.iloc[:,[0,2,3,4,5,9,10,11,12,13,14,15,16,17,18,19]],ded_df.iloc[:,:5],on='no', how='right')
ded_df= ded_df[['broadcast','m_code','p_code','p_name','p_group','exp_min','unit_price','total_price','sales','YEAR','MONTH','DAY','HOUR','MINUTE','SECOND','YEAR_DAY','DAY_NAME','DAY_NUM','code','no']]
ded_df.to_csv('ded_df.csv', encoding='utf-8-sig',index=False)

In [14]:
ded_df

,broadcast,m_code,p_code,p_name,p_group,exp_min,unit_price,total_price,sales,YEAR,MONTH,DAY,HOUR,MINUTE,SECOND,YEAR_DAY,DAY_NAME,DAY_NUM,code,no
0,2019-01-01 06:00:00,100346,201072,테이트 남성 셀린니트3종,의류,60.0,39900,12033000,302,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,0
1,2019-01-01 06:00:00,100346,201079,테이트 여성 셀린니트3종,의류,60.0,39900,20663000,518,2019,1,1,6,0,0,2019-01-01,Tuesday,2,0,1
2,2019-01-01 07:00:00,100305,200974,오모떼 레이스 파운데이션 브라,속옷,60.0,59000,47878000,812,2019,1,1,7,0,0,2019-01-01,Tuesday,2,1,6
3,2019-01-01 08:00:00,100808,202377,CERINI by PAT 남성 소프트 기모 릴렉스팬츠,의류,60.0,59900,99736000,1665,2019,1,1,8,0,0,2019-01-01,Tuesday,2,2,9
4,2019-01-01 09:00:00,100816,202404,보코 리버시블 무스탕,의류,60.0,79000,90973000,1151,2019,1,1,9,0,0,2019-01-01,Tuesday,2,3,12
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12505,2019-12-31 21:20:00,100155,200489,무이자 LG전자 매직스페이스 냉장고,가전,60.0,1499000,76448000,50,2019,12,31,21,20,0,2019-12-31,Tuesday,2,6857,35362
12506,2019-12-31 23:20:00,100448,201383,무이자쿠첸압력밥솥 10인용,주방,60.0,178000,64651000,363,2019,12,31,23,20,0,2019-12-31,Tuesday,2,6858,35367
12507,2019-12-31 23:20:00,100448,201390,일시불쿠첸압력밥솥 10인용,주방,60.0,168000,128803000,766,2019,12,31,23,20,0,2019-12-31,Tuesday,2,6858,35368
12508,2019-12-31 23:20:00,100448,201384,무이자쿠첸압력밥솥 6인용,주방,60.0,158000,16713000,106,2019,12,31,23,20,0,2019-12-31,Tuesday,2,6858,35369
